In [5]:
# Cell 0: Imports & helper utilities
import pandas as pd
import numpy as np
from pathlib import Path
from math import ceil
from datetime import timedelta

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 120)

print("Ready — imports done.")

Ready — imports done.


In [6]:
# Cell 1: Load admissions.csv (or use sample if file not present) and expand to one row per day (day_index)
csv_path = Path("admissions.csv")

admissions = pd.read_csv(csv_path, low_memory=False,
                            parse_dates=['admittime','dischtime','deathtime','edregtime','edouttime'])
print("Loaded admissions.csv from disk. Rows:", len(admissions))

# Ensure datetime types
admissions['admittime'] = pd.to_datetime(admissions['admittime'], errors='coerce')
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'], errors='coerce')

# If dischtime missing, fill with admittime (so at least one day is produced)
admissions['dischtime'] = admissions['dischtime'].fillna(admissions['admittime'])

# Function to expand a single admission row into day rows
def expand_admission_row(row):
    adm_date = row['admittime'].normalize().date()
    dis_date = row['dischtime'].normalize().date()
    n_days = (dis_date - adm_date).days + 1
    if n_days <= 0:
        n_days = 1
    rows = []
    for d in range(n_days):
        new = {
            'subject_id': row['subject_id'],
            'hadm_id': row['hadm_id'],
            'day_index': int(d),
            'admittime': row['admittime'],
            'dischtime': row['dischtime'],
            'deathtime': row['deathtime'],
            'admission_type': row.get('admission_type', np.nan),
            'admit_provider_id': row.get('admit_provider_id', np.nan),
            'admission_location': row.get('admission_location', np.nan),
            'discharge_location': row.get('discharge_location', np.nan),
            'insurance': row.get('insurance', np.nan),
            'language': row.get('language', np.nan),
            'marital_status': row.get('marital_status', np.nan),
            'race': row.get('race', np.nan),
            'edregtime': row.get('edregtime', pd.NaT),
            'edouttime': row.get('edouttime', pd.NaT),
            'hospital_expire_flag': row.get('hospital_expire_flag', np.nan)
        }
        rows.append(new)
    return rows

# Expand all admissions
expanded = []
for _, r in admissions.iterrows():
    expanded.extend(expand_admission_row(r))

merged_initial = pd.DataFrame(expanded)
# Convert types
merged_initial[['subject_id','hadm_id','day_index']] = merged_initial[['subject_id','hadm_id','day_index']].astype('Int64')

print("Expanded admissions -> rows:", merged_initial.shape[0])


Loaded admissions.csv from disk. Rows: 21987
Expanded admissions -> rows: 325800


In [7]:
# Cell 2: Attach icustays -> mark ICU presence per (subject_id, hadm_id, day_index)
# - Preserves merged_initial unchanged
# - Produces merged_with_icu with ICU columns filled when row_date falls inside an icu stay

icu_path = Path("icustays.csv")
if not icu_path.exists():
    raise FileNotFoundError(f"icustays.csv not found at {icu_path.resolve()}  -- put the file next to admissions.csv")

# load icustays (parse datetimes)
icustays = pd.read_csv(icu_path, low_memory=False, parse_dates=['intime','outtime'])

# normalize / required columns (tolerant if some optional cols missing)
for col in ['subject_id','hadm_id','intime','outtime']:
    if col not in icustays.columns:
        raise KeyError(f"Expected column '{col}' in icustays.csv but it is missing.")

# optional helpful columns: stay_id, first_careunit, last_careunit, los
optional_cols = ['stay_id','first_careunit','last_careunit','los']
for c in optional_cols:
    if c not in icustays.columns:
        icustays[c] = pd.NA  # create if missing so downstream code is simpler

# --- prepare merged copy (do not modify merged_initial in-place) ---
merged_with_icu = merged_initial.copy().reset_index(drop=False).rename(columns={'index':'row_id'})
# add ICU output columns (keeps original names minimal)
for col in ['stay_id_icu','icustay_intime','icustay_outtime','first_careunit_icu','last_careunit_icu','los_icu']:
    if col not in merged_with_icu.columns:
        merged_with_icu[col] = pd.NA

# compute the date of each merged row (admission day + day_index)
# assume admittime exists and day_index present
if 'admittime' not in merged_with_icu.columns:
    raise KeyError("merged_initial must contain 'admittime' column")

# ensure admittime is datetime
merged_with_icu['admittime'] = pd.to_datetime(merged_with_icu['admittime'], errors='coerce')
merged_with_icu['day_index_int'] = merged_with_icu['day_index'].fillna(0).astype(int)
merged_with_icu['row_date'] = merged_with_icu['admittime'].dt.normalize() + pd.to_timedelta(merged_with_icu['day_index_int'], unit='D')

# normalize icu intime/outtime to dates (fill missing outtime with intime)
icustays['intime'] = pd.to_datetime(icustays['intime'], errors='coerce')
icustays['outtime'] = pd.to_datetime(icustays['outtime'], errors='coerce').fillna(icustays['intime'])
icustays['intime_norm'] = icustays['intime'].dt.normalize()
icustays['outtime_norm'] = icustays['outtime'].dt.normalize()

# create a merged candidate set joining by subject_id & hadm_id (many-to-many)
# keep the icu's key cols for matching and assignment
icu_keep = ['subject_id','hadm_id','stay_id','intime','outtime','intime_norm','outtime_norm','first_careunit','last_careunit','los']
candidate = merged_with_icu.merge(icustays[icu_keep], on=['subject_id','hadm_id'], how='left', suffixes=('','_icu'))

# mark rows where row_date is within icu interval
mask_in_icu = (candidate['row_date'] >= candidate['intime_norm']) & (candidate['row_date'] <= candidate['outtime_norm'])
candidate['in_icu'] = mask_in_icu.fillna(False)

# For rows that match multiple ICU stays, pick the earliest ICU (by intime)
# keep only matches; then for duplicates keep the one with smallest intime
matched = candidate[candidate['in_icu']].copy()
if not matched.empty:
    # sort so earliest ICU intime appears first for each original row
    matched = matched.sort_values(by=['row_id','intime'])
    # pick first match per row_id
    first_matches = matched.groupby('row_id', as_index=False).first()
    # map the ICU fields back into merged_with_icu by row_id
    map_cols = {
        'stay_id':'stay_id_icu',
        'intime':'icustay_intime',
        'outtime':'icustay_outtime',
        'first_careunit':'first_careunit_icu',
        'last_careunit':'last_careunit_icu',
        'los':'los_icu'
    }
    for src, dst in map_cols.items():
        # build mapping series
        mapping = first_matches.set_index('row_id')[src]
        merged_with_icu.loc[merged_with_icu['row_id'].isin(mapping.index), dst] = merged_with_icu.loc[merged_with_icu['row_id'].isin(mapping.index), 'row_id'].map(mapping)
    assigned_count = len(first_matches)
else:
    assigned_count = 0

# cleanup helper columns
merged_with_icu = merged_with_icu.drop(columns=['day_index_int','row_date'])

print(f"ICU assignment complete. Rows where ICU info filled: {int(assigned_count)}")
# preview some assigned rows (if any)
print(merged_with_icu[merged_with_icu['stay_id_icu'].notna()].head(20))

# merged_with_icu is ready for next steps (icustays merged)
# note: merged_initial remains unchanged


ICU assignment complete. Rows where ICU info filled: 93109
    row_id  subject_id   hadm_id  day_index           admittime           dischtime           deathtime  \
7        7    10001217  24597018          2 2157-11-18 22:56:00 2157-11-25 18:00:00                 NaT   
8        8    10001217  24597018          3 2157-11-18 22:56:00 2157-11-25 18:00:00                 NaT   
48      48    10003400  23559586          6 2137-08-04 00:07:00 2137-09-02 17:05:00 2137-09-02 17:05:00   
49      49    10003400  23559586          7 2137-08-04 00:07:00 2137-09-02 17:05:00 2137-09-02 17:05:00   
50      50    10003400  23559586          8 2137-08-04 00:07:00 2137-09-02 17:05:00 2137-09-02 17:05:00   
51      51    10003400  23559586          9 2137-08-04 00:07:00 2137-09-02 17:05:00 2137-09-02 17:05:00   
55      55    10003400  23559586         13 2137-08-04 00:07:00 2137-09-02 17:05:00 2137-09-02 17:05:00   
56      56    10003400  23559586         14 2137-08-04 00:07:00 2137-09-02 17:05:00 2

In [ ]:
merged_with_icu

In [ ]:
merged_initial

In [ ]:
merged_initial.to_csv('merged_initial.csv')

In [ ]:
merged_with_icu.to_csv('merged_with_icu.csv')

In [8]:
merged_with_icu.head(100).to_csv('merged_with_icu_sample.csv')